In [1]:
"""Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.0.0+
Last tested with: v2.1.0
"""
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy
from spacy.util import minibatch, compounding
import pandas as pd
from spacy.util import decaying
from tqdm import tqdm
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def train(model=None, output_dir=None, n_iter=200, label="Loaded_Language"):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    dropout = decaying(0.6, 0.2, 1e-4)

    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in tqdm(range(n_iter)):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=next(dropout),  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

   
    output_dir = "spacy/model"
    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [33]:
from collections import defaultdict
import pathlib
import spacy
from spacy.tokens import Doc, Token
from torch import LongTensor
nlp = spacy.load('en')

from pathlib import Path

def read_data(directory, isLabels = True):
    ids = []
    texts = []
    labels = []
    for f in directory.glob('*.txt'):
        id = f.name.replace('article', '').replace('.txt','')
        ids.append(id)
        texts.append(f.read_text(encoding='utf-8'))
        if isLabels:
            labels.append(parse_label(f.as_posix().replace('.txt', '.task3.labels')))
    docs = list(nlp.pipe(texts))
    return ids, docs, labels

def parse_label(label_path):
    # idx, type, start, end
    labels = []
    f= Path(label_path)
    if not f.exists():
        return labels
    for line in open(label_path):
        parts = line.strip().split('\t')
        if ((parts[1]) == "Loaded_Language"):
            #print(parts[1])
            labels.append((int(parts[2]), int(parts[3]), parts[1]))
    return sorted(labels)

PROPAGANDA_TYPES = [
    "O",
    "Appeal_to_Authority",
    "Appeal_to_fear-prejudice",
    "Bandwagon",
    "Black-and-White_Fallacy",
    "Causal_Oversimplification",
    "Doubt",
    "Exaggeration,Minimisation",
    "Flag-Waving",
    "Loaded_Language",
    "Name_Calling,Labeling",
    "Obfuscation,Intentional_Vagueness,Confusion",
    "Red_Herring",
    "Reductio_ad_hitlerum",
    "Repetition",
    "Slogans",
    "Straw_Men",
    "Thought-terminating_Cliches",
    "Whataboutism",
    "B-Appeal_to_Authority",
    "B-Appeal_to_fear-prejudice",
    "B-Bandwagon",
    "B-Black-and-White_Fallacy",
    "B-Causal_Oversimplification",
    "B-Doubt",
    "B-Exaggeration,Minimisation",
    "B-Flag-Waving",
    "B-Loaded_Language",
    "B-Name_Calling,Labeling",
    "B-Obfuscation,Intentional_Vagueness,Confusion",
    "B-Red_Herring",
    "B-Reductio_ad_hitlerum",
    "B-Repetition",
    "B-Slogans",
    "B-Straw_Men",
    "B-Thought-terminating_Cliches",
    "B-Whataboutism"
]

PT2ID = {y: x for (x, y) in enumerate(PROPAGANDA_TYPES)}



In [41]:

directory = pathlib.Path('../task3/datasets-v5/tasks-2-3/test/')
ids, texts, labels = read_data(directory)

idnew = []
textnew = []
idn = []
TRAIN_DATA = []
for x, y, idx in zip(texts, labels, ids):
    if y:    
        idn.append(idx)
        textnew.append(y)
        new_dict = {"entities":y}
        tlist = [str(x), new_dict]
        TRAIN_DATA.append(tlist)


In [3]:
train(label = "Loaded_Language")

Created blank 'en' model


  0%|          | 1/200 [00:19<1:06:08, 19.94s/it]

Losses {'ner': 1048.464724261932}


  1%|          | 2/200 [00:38<1:04:20, 19.50s/it]

Losses {'ner': 268.042424036995}


...

Losses {'ner': 5.478014829023268}


100%|█████████▉| 199/200 [1:07:49<00:21, 21.17s/it]

Losses {'ner': 5.044219627136054}


100%|██████████| 200/200 [1:08:10<00:00, 21.15s/it]

Losses {'ner': 5.666175761633694}
Saved model to spacy/model


In [42]:
## Test Model
nlp2 = spacy.load('spacy/model')
counter = 0
id_csv = []
start_csv = []
end_csv = []
label_csv = []
label = "Loaded_Language"
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    currentid = idn[counter]
    for ent in doc.ents:
        x = ent
        a = x[0]
        start = a.idx
        a = x[-1]
        end = a.idx+len(a)
        start_csv.append(start)
        end_csv.append(end)
        id_csv.append(currentid)
        label_csv.append(label)
    counter = counter + 1
    #print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    #print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])



In [43]:
for text, _ in TRAIN_DATA:
    doc = nlp2(text)
    currentid = ids[counter]
    print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
    for ent in doc.ents:
        print (ent)
        print(ent[0].idx)
    break

Entities [('This severe and unprecedented', 'Loaded_Language')]
This severe and unprecedented
2476


In [44]:
df = {"ID": id_csv, "L": label_csv, "S": start_csv, "E": end_csv}
df = pd.DataFrame(df)
df.to_csv('../task3/predictions_spacy.csv', sep='\t', index=False, header=False) 

In [45]:
df

,ID,L,S,E
0,999001323,Loaded_Language,2476,2505
1,783774960,Loaded_Language,4391,4401
2,779309765,Loaded_Language,169,178
3,779309765,Loaded_Language,2511,2516
4,779309765,Loaded_Language,2957,2974
5,999001256,Loaded_Language,571,583
6,707772906,Loaded_Language,3571,3585
7,730019938,Loaded_Language,705,726
8,730019938,Loaded_Language,1080,1088
9,730019938,Loaded_Language,1360,1365
